In [1]:
import os
while not os.path.exists('.gitignore'):
	os.chdir(os.path.abspath(os.path.join('../')))
from common_imports import *

Defaulting to user installation because normal site-packages is not writeable


In [2]:
df = pd.read_parquet('_data/raw/juros-bancarios/combined.parquet')

In [3]:
linhas_para_remover = len(df) - 1000000

indices_para_remover = np.random.choice(df.index, size=linhas_para_remover, replace=False)

df = df.drop(indices_para_remover)

df = df.reset_index(drop=True)

In [4]:
df

,Segmento,Modalidade,InstituicaoFinanceira,TaxaJurosAoAno,data
0,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BRAZILIAN MORTGAGES CH S.A.,"18,50",2012-01-01
1,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BANCO INTER,"31,13",2012-01-01
2,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS REGULADAS ...,BCO BRADESCO S.A.,"12,06",2012-01-01
3,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS REGULADAS ...,BCO SANTANDER (BRASIL) S.A.,"12,84",2012-01-01
4,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BCO BRADESCO S.A.,"10,38",2012-01-01
...,...,...,...,...,...
999995,Pessoa Jurídica,Conta garantida - Pré-fixado,BCO DAYCOVAL S.A,"50,26",2024-09-17
999996,Pessoa Jurídica,Conta garantida - Pré-fixado,BCO DO EST. DE SE S.A.,"54,71",2024-09-17
999997,Pessoa Jurídica,Conta garantida - Pré-fixado,BCO RENDIMENTO S.A.,"68,10",2024-09-17
999998,Pessoa Jurídica,Conta garantida - Pré-fixado,BCO GUANABARA S.A.,"185,13",2024-09-17


In [5]:
df['TaxaJurosAoAno'] = df['TaxaJurosAoAno'].str.replace(',', '.').astype(float)

In [6]:
df['InstituicaoFinanceira'].value_counts()

InstituicaoFinanceira
BCO BRADESCO S.A.                     28765
BCO DO BRASIL S.A.                    28525
BCO SANTANDER (BRASIL) S.A.           27759
BCO DO ESTADO DO RS S.A.              26529
ITAÚ UNIBANCO S.A.                    24506
                                      ...  
BANIF INVESTIMENTO                        3
STATE STREET BR S.A. BCO COMERCIAL        3
BCO MORGAN STANLEY S.A.                   3
GOLDMAN SACHS DO BRASIL BM S.A            1
TRADIÇÃO                                  1
Name: count, Length: 270, dtype: int64

In [7]:
df.head()

,Segmento,Modalidade,InstituicaoFinanceira,TaxaJurosAoAno,data
0,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BRAZILIAN MORTGAGES CH S.A.,18.50,2012-01-01
1,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BANCO INTER,31.13,2012-01-01
2,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS REGULADAS ...,BCO BRADESCO S.A.,12.06,2012-01-01
3,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS REGULADAS ...,BCO SANTANDER (BRASIL) S.A.,12.84,2012-01-01
4,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO...,BCO BRADESCO S.A.,10.38,2012-01-01


In [8]:
df['Modalidade'].value_counts()

Modalidade
Crédito pessoal não-consignado - Pré-fixado                                                    92664
Cheque especial - Pré-fixado                                                                   69902
Crédito pessoal consignado privado - Pré-fixado                                                64250
Desconto de duplicatas - Pré-fixado                                                            63421
Capital de giro com prazo até 365 dias - Pré-fixado                                            59294
Aquisição de veículos - Pré-fixado                                                             58111
Cartão de crédito - rotativo total - Pré-fixado                                                54877
Capital de giro com prazo superior a 365 dias - Pré-fixado                                     52812
Crédito pessoal consignado público - Pré-fixado                                                52091
Aquisição de outros bens - Pré-fixado                                           

In [9]:
df['SubClassificacao'] = df['Modalidade'].apply(lambda x: x.split(' - ')[-1]).astype('category')
df['Modalidade'] = df['Modalidade'].apply(lambda x: x.rsplit(' - ', 1)[0]).astype('category')

In [10]:
df['Modalidade'].value_counts()

Modalidade
Capital de giro com prazo até 365 dias            95552
Crédito pessoal não-consignado                    92664
Conta garantida                                   90414
Capital de giro com prazo superior a 365 dias     90155
Cheque especial                                   69902
Crédito pessoal consignado privado                64250
Desconto de duplicatas                            63421
Aquisição de veículos                             58111
Cartão de crédito - rotativo total                54877
Crédito pessoal consignado público                52091
Aquisição de outros bens                          51853
Cartão de crédito - parcelado                     49861
Crédito pessoal consignado INSS                   47622
Adiantamento sobre contratos de câmbio (ACC)      44275
Desconto de cheques                               39270
Vendor                                            16948
Antecipação de faturas de cartão de crédito       11770
Arrendamento mercantil de veículos   

In [11]:
df['SubClassificacao'].value_counts()

SubClassificacao
Pré-fixado                                      835620
Pós-fixado referenciado em juros flutuantes     118030
Pós-fixado referenciado em moeda estrangeira     44275
PÓS-FIXADO REFERENCIADO EM TR                     1460
PRÉ-FIXADO                                         389
PÓS-FIXADO REFERENCIADO EM IPCA                    226
Name: count, dtype: int64

In [12]:
def transform_taxa(valor_taxa, periodo_desejado, periodo_atual):
	vt = Decimal(valor_taxa)
	pd = Decimal(periodo_desejado)
	pa = Decimal(periodo_atual)
 
	a = 1 + (vt / 100)
	b = pd / pa
	c = a ** b
	d = (c - 1) * 100

	return d

In [13]:
def trim_decimal(n):
	valor = Decimal(str(n).rstrip('0').rstrip('.'))
	return valor.quantize(Decimal('1.0000'), rounding=ROUND_HALF_UP)

In [14]:
df

,Segmento,Modalidade,InstituicaoFinanceira,TaxaJurosAoAno,data,SubClassificacao
0,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BRAZILIAN MORTGAGES CH S.A.,18.50,2012-01-01,PRÉ-FIXADO
1,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BANCO INTER,31.13,2012-01-01,PRÉ-FIXADO
2,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS REGULADAS,BCO BRADESCO S.A.,12.06,2012-01-01,PRÉ-FIXADO
3,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS REGULADAS,BCO SANTANDER (BRASIL) S.A.,12.84,2012-01-01,PRÉ-FIXADO
4,Pessoa Física,FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO,BCO BRADESCO S.A.,10.38,2012-01-01,PÓS-FIXADO REFERENCIADO EM TR
...,...,...,...,...,...,...
999995,Pessoa Jurídica,Conta garantida,BCO DAYCOVAL S.A,50.26,2024-09-17,Pré-fixado
999996,Pessoa Jurídica,Conta garantida,BCO DO EST. DE SE S.A.,54.71,2024-09-17,Pré-fixado
999997,Pessoa Jurídica,Conta garantida,BCO RENDIMENTO S.A.,68.10,2024-09-17,Pré-fixado
999998,Pessoa Jurídica,Conta garantida,BCO GUANABARA S.A.,185.13,2024-09-17,Pré-fixado


In [15]:
df['TaxaJurosAoMes'] = df.apply(lambda x: trim_decimal(transform_taxa(x['TaxaJurosAoAno'], 1, 12)), axis=1).astype(float)

In [16]:
SUBCLASSIFICACAO_REPLACE_MAP = {
	'PÓS-FIXADO': 'Pós-fixado',
	'PRÉ-FIXADO': 'Pré-fixado',
	'PÓS-FIXADO REFERENCIADO EM IPCA': 'Pós-fixado referenciado em IPCA',
	'PÓS-FIXADO REFERENCIADO EM TR': 'Pós-fixado referenciado em TR',
}
df['SubClassificacao'] = df['SubClassificacao'].replace(SUBCLASSIFICACAO_REPLACE_MAP).astype('category')

In [17]:
df['SubClassificacao'].value_counts()

SubClassificacao
Pré-fixado                                      836009
Pós-fixado referenciado em juros flutuantes     118030
Pós-fixado referenciado em moeda estrangeira     44275
Pós-fixado referenciado em TR                     1460
Pós-fixado referenciado em IPCA                    226
Name: count, dtype: int64

In [18]:
MODALIDADE_REPLACE_MAP = {
	'FINANCIAMENTO IMOBILIÁRIO COM TAXAS DE MERCADO': 'Financiamento imobiliário com taxas de mercado',
	'FINANCIAMENTO IMOBILIÁRIO COM TAXAS REGULADAS': 'Financiamento imobiliário com taxas reguladas',
}
df['Modalidade'] = df['Modalidade'].replace(MODALIDADE_REPLACE_MAP).astype('category')

In [19]:
df['Modalidade'].value_counts()

Modalidade
Capital de giro com prazo até 365 dias            95552
Crédito pessoal não-consignado                    92664
Conta garantida                                   90414
Capital de giro com prazo superior a 365 dias     90155
Cheque especial                                   69902
Crédito pessoal consignado privado                64250
Desconto de duplicatas                            63421
Aquisição de veículos                             58111
Cartão de crédito - rotativo total                54877
Crédito pessoal consignado público                52091
Aquisição de outros bens                          51853
Cartão de crédito - parcelado                     49861
Crédito pessoal consignado INSS                   47622
Adiantamento sobre contratos de câmbio (ACC)      44275
Desconto de cheques                               39270
Vendor                                            16948
Antecipação de faturas de cartão de crédito       11770
Arrendamento mercantil de veículos   

In [20]:
MODALIDADE_REPLACE_MAP = {
	'Pessoa Física': 'PF',
	'Pessoa Jurídica': 'PJ',
}
df['Segmento'] = df['Segmento'].replace(MODALIDADE_REPLACE_MAP).astype('category')

In [21]:
df['Segmento'].value_counts()

Segmento
PF    530161
PJ    469839
Name: count, dtype: int64

In [22]:
df['InstituicaoFinanceira'].value_counts()

InstituicaoFinanceira
BCO BRADESCO S.A.                     28765
BCO DO BRASIL S.A.                    28525
BCO SANTANDER (BRASIL) S.A.           27759
BCO DO ESTADO DO RS S.A.              26529
ITAÚ UNIBANCO S.A.                    24506
                                      ...  
BANIF INVESTIMENTO                        3
STATE STREET BR S.A. BCO COMERCIAL        3
BCO MORGAN STANLEY S.A.                   3
GOLDMAN SACHS DO BRASIL BM S.A            1
TRADIÇÃO                                  1
Name: count, Length: 270, dtype: int64

In [23]:
df.dtypes

Segmento                 category
Modalidade               category
InstituicaoFinanceira      object
TaxaJurosAoAno            float64
data                       object
SubClassificacao         category
TaxaJurosAoMes            float64
dtype: object

In [24]:
df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d')

In [25]:
df = df[df['data'] > (datetime.now() - timedelta(days=7*365))]

In [26]:
df.rename(columns={
    'Segmento': 'ct_classificacao',
    'Modalidade': 'ct_modalidade',
    'InstituicaoFinanceira': 'ct_instituicao_financeira',
    'SubClassificacao': 'ct_indexador_modalidade',
    'TaxaJurosAoAno': 'vl_taxa_juros_ano',
    'TaxaJurosAoMes': 'vl_taxa_juros_mes',
	'data': 'dt_referencia',
}, inplace=True)

/tmp/ipykernel_72094/226268388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={


In [27]:
df.dtypes

ct_classificacao                   category
ct_modalidade                      category
ct_instituicao_financeira            object
vl_taxa_juros_ano                   float64
dt_referencia                datetime64[ns]
ct_indexador_modalidade            category
vl_taxa_juros_mes                   float64
dtype: object

In [28]:
df.to_parquet('_data/trusted/juros-bancarios/df.parquet', index=False, engine='fastparquet')
df.to_parquet('_data/refined/juros-bancarios/df.parquet', index=False, engine='fastparquet')

In [29]:
df.to_csv('_data/trusted/juros-bancarios/df.csv', index=False)
df.to_csv('_data/refined/juros-bancarios/df.csv', index=False)